#### Note

In [ ]:
# pd.set_option('display.max_rows', 100) # 최대 display option set
# pd.reset_option('display.max_rows') # 최대 display option reset

# from datetime import timedelta
# datetime.datetime(2019,12,31) - datetime.datetime(2008,1,1)

#### Import

In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### ABSI 정의
체형지수(ABSI) = 허리둘레 / (BMI × 2/3) × (신장×1/2)

#### 98 - 19 기본 DB concat

필요변수 목록   
HE_ht         
HE_wt        
HE_BMI         
HE_wc

In [2]:
hn19_all = pd.read_sas('D:\\국민건강영양조사\\1998-2019 기본DB\\hn19_all.sas7bdat', format='sas7bdat')
hn19_all

,mod_d,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,...,LF_S10,LF_S11,LF_S12,LF_S13,LF_S14,LF_S14_1,LF_S15,LF_S16,LF_SECUR,LF_SECUR_G
0,b'2021.02.26.',b'A501215301',b'A5012153',2019.0,1.0,1.0,1.0,b'A501',1.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b'2021.02.26.',b'A501215303',b'A5012153',2019.0,1.0,1.0,1.0,b'A501',1.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b'2021.02.26.',b'A501220401',b'A5012204',2019.0,1.0,1.0,1.0,b'A501',1.0,53.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
3,b'2021.02.26.',b'A501220402',b'A5012204',2019.0,1.0,1.0,1.0,b'A501',2.0,50.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
4,b'2021.02.26.',b'A501220403',b'A5012204',2019.0,1.0,1.0,1.0,b'A501',1.0,16.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8105,b'2021.02.26.',b'R504381902',b'R5043819',2019.0,8.0,1.0,2.0,b'R504',2.0,48.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8106,b'2021.02.26.',b'R504393801',b'R5043938',2019.0,8.0,1.0,2.0,b'R504',1.0,47.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8107,b'2021.02.26.',b'R504393802',b'R5043938',2019.0,8.0,1.0,2.0,b'R504',2.0,43.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8108,b'2021.02.26.',b'R504393803',b'R5043938',2019.0,8.0,1.0,2.0,b'R504',2.0,16.0,...,3.0,3.0,3.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0


In [8]:
hn19_all['age'].describe()

count    8110.000000
mean       43.854131
std        22.548851
min         1.000000
25%        25.250000
50%        46.000000
75%        62.000000
max        80.000000
Name: age, dtype: float64